In [174]:
import json
import networkx as nx
import numpy as np
import plotly.graph_objects as go
from networkx.drawing.nx_agraph import graphviz_layout

In [175]:
filename = "example-2.json"

In [176]:
with open(filename) as f:
    data = json.load(f)

In [177]:
graph = nx.Graph(directed=True)
graph.add_nodes_from(map(lambda scene: scene['scene_id'], data['scenes']))
graph.add_edges_from(
    (choice['next_scene'], scene['scene_id'])
    for scene in data['scenes']
    for choice in scene['choices']
)

In [178]:
# Генерируем координаты вершин графа
pos = graphviz_layout(graph, prog='dot', args="-Grankdir=LR")

coords = np.array(list(pos.values()))


# Извлекаем информацию о положении вершин и рёбер графа
node_x = coords[:, 0] # Заполняем списки вершин
node_y = coords[:, 1]
node_text = list(graph.nodes())
edge_x, edge_y = [], []

for edge in graph.edges():
  n_1 = node_text.index(edge[0])
  n_2 = node_text.index(edge[1])
  edge_x.extend([node_x[n_1], node_x[n_2], None])
  edge_y.extend([node_y[n_1], node_y[n_2], None])

In [179]:
# Визуализируем рёбра графа (основные линии)
edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines',
)

# # Визуализируем вершины графа
# node_trace = go.Scatter(
#     x=node_x, y=node_y,
#     mode='markers+text',
#     text=node_text,
#     textposition="middle center",
#     textfont=dict(size=9),
#     hoverinfo='text',
#     marker=dict(
#         symbol='square',
#         color='lightblue',
#         size=40,
#         line_width=2
#     )
# )

# Создаём фигуру
fig = go.Figure(
    data=[edge_trace],
    layout=go.Layout(
        title='Scene Graph (Directed)',
        showlegend=False,
        hovermode='closest',
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        height=1000, 
        width=1000
    )
)

for i, (x, y, text) in enumerate(zip(node_x, node_y, node_text)):
    fig.add_annotation(
        x=x,
        y=y,
        text=text,
        showarrow=False,
        font=dict(size=9, color='black'),
        align='center',
        bordercolor='darkblue',
        borderwidth=1,
        borderpad=4,
        bgcolor='lightblue',
        width=120,  # Ширина прямоугольника (пиксели)
        height=30, # Высота прямоугольника (пиксели)
        xanchor='center',
        yanchor='middle'
    )

fig.show()